# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
"Lng",
"Lat", 
geo = True,
titles = "OSM",
size = "Humidity",
color = "City",
)

# Display the map
city_plot

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Humidity'] < 45)]
city_data_df = city_data_df[(city_data_df['Max Temp']>= 20) & (city_data_df['Max Temp'] <= 25)]


# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
86,86,rafaela,-31.2503,-61.4867,21.10,42,100,1.79,AR,1666108109
102,102,rio gallegos,-51.6226,-69.2181,20.03,26,75,8.23,AR,1666108296
106,106,palencia,42.4167,-4.5000,24.40,30,40,8.66,ES,1666108219
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102
296,296,jalu,29.0331,21.5482,22.33,37,25,3.89,LY,1666108413
313,313,tandil,-37.3217,-59.1332,21.63,40,75,6.10,AR,1666108422
328,328,viseu,40.6610,-7.9097,22.70,39,34,6.13,PT,1666108429
464,464,vryburg,-26.9566,24.7284,23.70,44,94,3.87,ZA,1666108505


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
86,rafaela,AR,-31.2503,-61.4867,42,
102,rio gallegos,AR,-51.6226,-69.2181,26,
106,palencia,ES,42.4167,-4.5000,30,
239,rawson,AR,-43.3002,-65.1023,25,
296,jalu,LY,29.0331,21.5482,37,
313,tandil,AR,-37.3217,-59.1332,40,
328,viseu,PT,40.6610,-7.9097,39,
464,vryburg,ZA,-26.9566,24.7284,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accomodation.hotel',
    'apiKey' : geoapify_key,
    'limit': 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
rafaela - nearest hotel: No hotel found
rio gallegos - nearest hotel: No hotel found
palencia - nearest hotel: No hotel found
rawson - nearest hotel: No hotel found
jalu - nearest hotel: No hotel found
tandil - nearest hotel: No hotel found
viseu - nearest hotel: No hotel found
vryburg - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
86,rafaela,AR,-31.2503,-61.4867,42,No hotel found
102,rio gallegos,AR,-51.6226,-69.2181,26,No hotel found
106,palencia,ES,42.4167,-4.5000,30,No hotel found
239,rawson,AR,-43.3002,-65.1023,25,No hotel found
296,jalu,LY,29.0331,21.5482,37,No hotel found
313,tandil,AR,-37.3217,-59.1332,40,No hotel found
328,viseu,PT,40.6610,-7.9097,39,No hotel found
464,vryburg,ZA,-26.9566,24.7284,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_plot = city_data_df.hvplot.points(
"Lng",
"Lat", 
geo = True,
titles = "OSM",
size = "Humidity",
color = "City",
hover_cols = ['Country', 'Hotel Name']
)

# Display the map
hotel_plot

:Points   [Lng,Lat]   (City,Humidity,Country)